In [1]:
!pip3 install tomlkit
!pip3 install -U neo4j

     ---------------------------------------- 0.0/198.0 kB ? eta -:--:--
     -- ------------------------------------- 10.2/198.0 kB ? eta -:--:--
     ------- ----------------------------- 41.0/198.0 kB 487.6 kB/s eta 0:00:01
     ---------------------- ------------- 122.9/198.0 kB 901.1 kB/s eta 0:00:01
     ------------------------------------ 198.0/198.0 kB 998.7 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for neo4j: filename=neo4j-5.18.0-py3-none-any.whl size=273889 sha256=5f2d20da7624f690440ce0a43031f4848015682c2c27f4af3b13a68e5e99ec32
  Stored in directory: c:\u

In [18]:
from neo4j import GraphDatabase

database_name = "neo4j"
username = "neo4j"
password = "apan5400"
uri = "bolt://localhost:7687/" + database_name

driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

print("Successfully connected to Neo4j!")

Successfully connected to Neo4j!


In [14]:
import os
import urllib

# Define the path to the CSV file
datapath = "C:/Users/rober/Downloads/companies.csv"
print("file:///{0}".format(datapath))

# Cypher query to import data from CSV into Neo4j
query = ("LOAD CSV WITH HEADERS FROM 'file:///{0}' AS line".format(datapath)
+ """
CREATE (company:Company { \
                       name: line.company, \
                  employees: toInteger(line.employees),\
                       year: toInteger(line.founded), \
                    revenue: toFloat(line.revenue) \
                            } \
            )
MERGE  (city:City {name: line.city})
MERGE  (state:State {name: line.state})
MERGE  (country:Country {name: line.country})
CREATE (company)-[:HEADQUARTERED_IN]->(city)
CREATE (city)-[:LOCATED_IN]->(state)
CREATE (state)-[:PART_OF]->(country)
"""
)

# Function to import data from CSV into Neo4j
def import_csv_data(tx, query):
    result = tx.run(query)
    return result

# Connect to Neo4j and import data
driver = GraphDatabase.driver(uri, auth=(username, password))
with driver.session() as session:
    result = session.write_transaction(import_csv_data, query)
    print("All companies are imported from the CSV file!")

file:///C:/Users/rober/Downloads/companies.csv


C:\Users\rober\AppData\Local\Temp\ipykernel_22304\2618663068.py:35: DeprecationWarning: write_transaction has been renamed to execute_write
  result = session.write_transaction(import_csv_data, query)


All companies are imported from the CSV file!


In [21]:
# 1.Which companies are located in the state of California (via 2nd degree relation)?
query1 = """
MATCH (company:Company)-[:HEADQUARTERED_IN]->(:City)-[:LOCATED_IN]->(:State {name: 'California'})
RETURN company.name
"""
result1 = session.run(query1)
for record in result1:
    print(record["company.name"])

Alphabet
Apple


In [22]:
# 2.Which companies have been founded in the last quarter of the 20th century [1975-2000]? 
query2 = """
MATCH (company:Company)
WHERE company.year >= 1975 AND company.year <= 2000
RETURN company.name
"""
result2 = session.run(query2)
for record in result2:
    print(record["company.name"])

Apple
Microsoft
Amazon
Alphabet
Alibaba Group
China National Petroleum Corporation


In [34]:
# 3.What is the average annual revenue for all companies in the USA?
query3 = """
MATCH (company:Company)-[:HEADQUARTERED_IN]->(:City)-[:LOCATED_IN]->(:State)-[:PART_OF]->(:Country {name: 'USA'})
RETURN AVG(company.revenue) AS average_revenue
"""
result3 = session.run(query3)
for record in result3:
    print(record["average_revenue"])

294.176


In [36]:
# 4.Which companies are headquartered in Japan (via 3rd degree relation)?
query4 = """
MATCH (company:Company)-[:HEADQUARTERED_IN]->(:City)-[:LOCATED_IN]->(:State)-[:PART_OF]->(:Country {name: 'Japan'})
RETURN company.name AS company_name
"""
result4 = session.run(query4)
for record in result4:
    print(record["company_name"])

Sony Group
Toyota Motor


In [40]:
# Companies in the USA
query5 = """
MATCH (company:Company)-[:HEADQUARTERED_IN]->(:City)-[:LOCATED_IN]->(:State)-[:PART_OF]->(:Country {name: 'USA'})
return company.name
"""
result5 = session.run(query5)
for record in result5:
    print(record["company.name"])

Amazon
Microsoft
Alphabet
Apple
Tesla


In [42]:
# Companies in Germany
query6 = """
MATCH (company:Company)-[:HEADQUARTERED_IN]->(:City)-[:LOCATED_IN]->(:State)-[:PART_OF]->(:Country {name: 'Germany'})
return company.name
"""
result6 = session.run(query6)
for record in result6:
    print(record["company.name"])

Allianz
Mercedes-Benz Group


In [45]:
# 5.What is the total number of employees of all companies in the USA and Germany (via 3rd degree relation)?
query7 = """
MATCH (company:Company)
WHERE (company)-[:HEADQUARTERED_IN]->(:City)-[:LOCATED_IN]->(:State)-[:PART_OF]->(:Country {name: 'USA'})
   OR (company)-[:HEADQUARTERED_IN]->(:City)-[:LOCATED_IN]->(:State)-[:PART_OF]->(:Country {name: 'Germany'})
RETURN SUM(company.employees) AS total_employees
"""
result7 = session.run(query7)
for record in result7:
    print(record["total_employees"])

2571925
